In [ ]:
%%capture output
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install -q selenium

In [ ]:
# Lets import everything
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time

In [ ]:
# Sadly this page does not like being scrapped :/
!pip install undetected-chromedriver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 4.0 MB/s eta 0:00:00
  Created wheel for undetected-chromedriver: filename=undetected_chromedriver-3.5.5-py3-none-any.whl size=47048 sha256=41fc5dda517262626c07f999ce99cd2e1210ac5da9e27bbbec87c7d74862f0fa
  Stored in directory: /root/.cache/pip/wheels/cf/a1/db/e1275b6f7259aacd6b045f8bfcb1fcbc93827a3916ba55d5b7
Successfully built undetected-chromedriver


In [ ]:
import undetected_chromedriver as uc

# Create an instance of the Chrome browser driver
driver = uc.Chrome(version_main=90,headless = True,use_subprocess=False)
time.sleep(3)

# Load the website
driver.get("https://www.cityguideny.com/restaurants/")

from IPython.display import HTML,display
display(HTML(driver.page_source)) # To make sure it works

,


In [ ]:
# Okay it works! Let's extract the name, phone, and website for the retaurants listed here
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
import re

def get_restaurant_info(url):
    restaurant_info = []

    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = uc.Chrome(version_main=90,headless = True,use_subprocess=False)

    try:
        driver.get(url)
        wait = WebDriverWait(driver, 10)

        # Find all restaurant sections
        restaurant_sections = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class,'side-box')]")))

        for section in restaurant_sections:
            try:
                # Extract the restaurant name
                name_element = section.find_element(By.XPATH, ".//h5/a")
                restaurant_name = name_element.text
            except Exception as e:
                # If there's an error (e.g., element not found), skip to the next section
                continue

            try:
                # Extract the phone number
                p_elements = section.find_elements(By.XPATH, ".//p")
                phone_number = None
                for p_element in p_elements:
                    text = p_element.text
                    match = re.search(r'\d{3}-\d{3}-\d{4}', text)
                    if match:
                        phone_number = match.group()
                        break  # Stop searching once a phone number is found
            except Exception as e:
                phone_number = None

            try:
                # Extract the restaurant's website
                website_element = section.find_element(By.XPATH, ".//p/a")
                website = website_element.get_attribute('href')
            except Exception as e:
                website = None

            if restaurant_name:
                restaurant_info.append({'Name': restaurant_name, 'Phone': phone_number, 'Website': website})
    finally:
        driver.quit()

    return restaurant_info

url = "https://www.cityguideny.com/restaurants/"
restaurant_info = get_restaurant_info(url)
print(restaurant_info)

[{'Name': 'Benjamin Steakhouse', 'Phone': '212-297-9177', 'Website': 'https://www.cityguideny.com/linktrack.cfm?id=1351&table=Restaurants'}, {'Name': 'Park Ave Kitchen by David Burke', 'Phone': '646-847-4166', 'Website': 'https://www.cityguideny.com/linktrack.cfm?id=6140&table=Restaurants'}, {'Name': 'Raising Cane’s Chicken Fingers', 'Phone': None, 'Website': 'https://www.cityguideny.com/linktrack.cfm?id=6143&table=Restaurants'}, {'Name': 'The Sea Fire Grill', 'Phone': '212-935-3785', 'Website': 'https://www.cityguideny.com/linktrack.cfm?id=5867&table=Restaurants'}, {'Name': 'Benjamin Prime', 'Phone': '212-338-0818', 'Website': 'https://www.cityguideny.com/linktrack.cfm?id=6048&table=Restaurants'}, {'Name': 'Cafe Fiorello', 'Phone': '212-595-5330', 'Website': 'https://www.cityguideny.com/linktrack.cfm?id=6127&table=Restaurants'}, {'Name': 'Redeye Grill', 'Phone': '212-541-9000', 'Website': 'https://www.cityguideny.com/linktrack.cfm?id=6123&table=Restaurants'}, {'Name': 'The Palm Restau

In [ ]:
# Now that we extracted the data lets display it in a table
import pandas as pd

data = restaurant_info
df = pd.DataFrame(data)

df

,Name,Phone,Website
0,Benjamin Steakhouse,212-297-9177,https://www.cityguideny.com/linktrack.cfm?id=1...
1,Park Ave Kitchen by David Burke,646-847-4166,https://www.cityguideny.com/linktrack.cfm?id=6...
2,Raising Cane’s Chicken Fingers,None,https://www.cityguideny.com/linktrack.cfm?id=6...
3,The Sea Fire Grill,212-935-3785,https://www.cityguideny.com/linktrack.cfm?id=5...
4,Benjamin Prime,212-338-0818,https://www.cityguideny.com/linktrack.cfm?id=6...
5,Cafe Fiorello,212-595-5330,https://www.cityguideny.com/linktrack.cfm?id=6...
6,Redeye Grill,212-541-9000,https://www.cityguideny.com/linktrack.cfm?id=6...
7,The Palm Restaurant,212-333-7256,https://www.cityguideny.com/linktrack.cfm?id=1...
8,Gatsby's Landing,212-597-2850,https://www.cityguideny.com/linktrack.cfm?id=6...
9,Bond 45,212-869-4545,https://www.cityguideny.com/linktrack.cfm?id=6...


In [ ]:
print(type(restaurant_info))

In [ ]:
# We should add a few more columns that would help when choosing which of these places to go to. We should include the
# location of each restaurant, rating, and type of cusine. To get this information we can use Yelp's business search API

def get_yelp_business_info(api_key, restaurant_name, location='New York City'):

    headers = {
        'Authorization': f'Bearer {api_key}',
    }
    params = {
        'term': restaurant_name,
        'location': location,
        'limit': 1
    }
    response = requests.get('https://api.yelp.com/v3/businesses/search', headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        businesses = data.get('businesses')
        if businesses:
            business = businesses[0]
            return {
                'Location': ', '.join(business['location']['display_address']),
                'Rating': business['rating'],
                'Type of Cuisine': ', '.join([category['title'] for category in business['categories']]) if business['categories'] else 'N/A',
                'Price Point': business.get('price', 'N/A')  # Extracting price point
            }
    else:
        print(f"Error fetching data from Yelp API: {response.status_code}")
        return {}

api_key = 'xeoszcm_TE2A8kbVLZRgNLplKsO6RqVoMt3T_-GHtqhB6FBiG-5bBhvsc3keZCg-CuMNMWNV6d4ALU9MngDBBSrjGX6AFVmtxHfm0GUYnrYXo0XJ0twx4RyZtZ3yZXYx'
enhanced_restaurant_info = []

for restaurant in restaurant_info:
    yelp_info = get_yelp_business_info(api_key, restaurant['Name'], location='New York')
    if yelp_info:
        # Combine the original info with the new Yelp info
        combined_info = {**restaurant, **yelp_info}
        enhanced_restaurant_info.append(combined_info)
    else:
        # In case of an API failure, append the original data
        enhanced_restaurant_info.append(restaurant)

df = pd.DataFrame(enhanced_restaurant_info)
df

,Name,Phone,Website,Location,Rating,Type of Cuisine,Price Point
0,Benjamin Steakhouse,212-297-9177,https://www.cityguideny.com/linktrack.cfm?id=1...,"52 E 41st St, New York City, NY 10017",4.3,"Steakhouses, Seafood, Bars",$$$$
1,Park Ave Kitchen by David Burke,646-847-4166,https://www.cityguideny.com/linktrack.cfm?id=6...,"514 Lexington Ave 48th St, New York, NY 10017",4.3,"New American, Breakfast & Brunch",N/A
2,Raising Cane’s Chicken Fingers,None,https://www.cityguideny.com/linktrack.cfm?id=6...,"79 Chrystie St, New York, NY 10002",4.5,"Fast Food, Cantonese",$
3,The Sea Fire Grill,212-935-3785,https://www.cityguideny.com/linktrack.cfm?id=5...,"158 E 48th St, New York, NY 10017",4.5,"Seafood, Steakhouses, Bars",$$$
4,Benjamin Prime,212-338-0818,https://www.cityguideny.com/linktrack.cfm?id=6...,"23 E 40th St, New York, NY 10016",4.4,"Seafood, Steakhouses, Bars",$$$$
5,Cafe Fiorello,212-595-5330,https://www.cityguideny.com/linktrack.cfm?id=6...,"1900 Broadway, New York, NY 10023",3.6,"Italian, Pizza, American",$$$
6,Redeye Grill,212-541-9000,https://www.cityguideny.com/linktrack.cfm?id=6...,"890 7th Ave, New York, NY 10019",3.7,"New American, Seafood, Cocktail Bars",$$$
7,The Palm Restaurant,212-333-7256,https://www.cityguideny.com/linktrack.cfm?id=1...,"250 W 50th St, New York, NY 10019",3.6,"Steakhouses, Seafood",$$$
8,Gatsby's Landing,212-597-2850,https://www.cityguideny.com/linktrack.cfm?id=6...,"120 West 44th St, New York, NY 10036",4.3,New American,$$$
9,Bond 45,212-869-4545,https://www.cityguideny.com/linktrack.cfm?id=6...,"221 W 46th St, New York, NY 10036",3.7,"Italian, Pizza, New American",$$$


In [ ]:
# We have some null values in the the Price Category. We can manually add them using some info from Google
# "Price per Person." $= under $10. $$=11–30. $$$=31–60. $$$$= over $61.
updates = {
    "Park Ave Kitchen by David Burke" : {"Price Point": "$$$"},
    "USA Brooklyn Delicatessen": {"Price Point": "$$"},
    "Sugar Factory Times Square": {"Price Point": "$$$"},
    "The Consulate Midtown": {"Price Point": "$$$$"},
    "Café Paradiso": {"Phone": "+12128698800", "Price Point": "$$"},
}

for title, data in updates.items():
    for key, value in data.items():
        df.loc[df['Name'] == title, key] = value

# Lets also rename Title to Name
df = df.rename(columns={'Title': 'Name'})
df

,Name,Phone,Website,Location,Rating,Type of Cuisine,Price Point
0,Benjamin Steakhouse,212-297-9177,https://www.cityguideny.com/linktrack.cfm?id=1...,"52 E 41st St, New York City, NY 10017",4.3,"Steakhouses, Seafood, Bars",$$$$
1,Park Ave Kitchen by David Burke,646-847-4166,https://www.cityguideny.com/linktrack.cfm?id=6...,"514 Lexington Ave 48th St, New York, NY 10017",4.3,"New American, Breakfast & Brunch",$$$
2,Raising Cane’s Chicken Fingers,None,https://www.cityguideny.com/linktrack.cfm?id=6...,"79 Chrystie St, New York, NY 10002",4.5,"Fast Food, Cantonese",$
3,The Sea Fire Grill,212-935-3785,https://www.cityguideny.com/linktrack.cfm?id=5...,"158 E 48th St, New York, NY 10017",4.5,"Seafood, Steakhouses, Bars",$$$
4,Benjamin Prime,212-338-0818,https://www.cityguideny.com/linktrack.cfm?id=6...,"23 E 40th St, New York, NY 10016",4.4,"Seafood, Steakhouses, Bars",$$$$
5,Cafe Fiorello,212-595-5330,https://www.cityguideny.com/linktrack.cfm?id=6...,"1900 Broadway, New York, NY 10023",3.6,"Italian, Pizza, American",$$$
6,Redeye Grill,212-541-9000,https://www.cityguideny.com/linktrack.cfm?id=6...,"890 7th Ave, New York, NY 10019",3.7,"New American, Seafood, Cocktail Bars",$$$
7,The Palm Restaurant,212-333-7256,https://www.cityguideny.com/linktrack.cfm?id=1...,"250 W 50th St, New York, NY 10019",3.6,"Steakhouses, Seafood",$$$
8,Gatsby's Landing,212-597-2850,https://www.cityguideny.com/linktrack.cfm?id=6...,"120 West 44th St, New York, NY 10036",4.3,New American,$$$
9,Bond 45,212-869-4545,https://www.cityguideny.com/linktrack.cfm?id=6...,"221 W 46th St, New York, NY 10036",3.7,"Italian, Pizza, New American",$$$


In [ ]:
# Update Phone column by formatting the phone numbers
df['Phone'] = df['Phone'].str.replace(r'^(?:\+1)?(\d{3})(\d{3})(\d{4})$', r'(\1) \2-\3', regex=True)
df['Phone'] = df['Phone'].str.replace(r'^(?:\+1)?(\d{3})-(\d{3})-(\d{4})$', r'(\1) \2-\3', regex=True)
df

,Name,Phone,Website,Location,Rating,Type of Cuisine,Price Point
0,Benjamin Steakhouse,(212) 297-9177,https://www.cityguideny.com/linktrack.cfm?id=1...,"52 E 41st St, New York City, NY 10017",4.3,"Steakhouses, Seafood, Bars",$$$$
1,Park Ave Kitchen by David Burke,(646) 847-4166,https://www.cityguideny.com/linktrack.cfm?id=6...,"514 Lexington Ave 48th St, New York, NY 10017",4.3,"New American, Breakfast & Brunch",$$$
2,Raising Cane’s Chicken Fingers,None,https://www.cityguideny.com/linktrack.cfm?id=6...,"79 Chrystie St, New York, NY 10002",4.5,"Fast Food, Cantonese",$
3,The Sea Fire Grill,(212) 935-3785,https://www.cityguideny.com/linktrack.cfm?id=5...,"158 E 48th St, New York, NY 10017",4.5,"Seafood, Steakhouses, Bars",$$$
4,Benjamin Prime,(212) 338-0818,https://www.cityguideny.com/linktrack.cfm?id=6...,"23 E 40th St, New York, NY 10016",4.4,"Seafood, Steakhouses, Bars",$$$$
5,Cafe Fiorello,(212) 595-5330,https://www.cityguideny.com/linktrack.cfm?id=6...,"1900 Broadway, New York, NY 10023",3.6,"Italian, Pizza, American",$$$
6,Redeye Grill,(212) 541-9000,https://www.cityguideny.com/linktrack.cfm?id=6...,"890 7th Ave, New York, NY 10019",3.7,"New American, Seafood, Cocktail Bars",$$$
7,The Palm Restaurant,(212) 333-7256,https://www.cityguideny.com/linktrack.cfm?id=1...,"250 W 50th St, New York, NY 10019",3.6,"Steakhouses, Seafood",$$$
8,Gatsby's Landing,(212) 597-2850,https://www.cityguideny.com/linktrack.cfm?id=6...,"120 West 44th St, New York, NY 10036",4.3,New American,$$$
9,Bond 45,(212) 869-4545,https://www.cityguideny.com/linktrack.cfm?id=6...,"221 W 46th St, New York, NY 10036",3.7,"Italian, Pizza, New American",$$$


In [ ]:
df_dropped = df.drop(index=2)
df_dropped

,Name,Phone,Website,Location,Rating,Type of Cuisine,Price Point
0,Benjamin Steakhouse,(212) 297-9177,https://www.cityguideny.com/linktrack.cfm?id=1...,"52 E 41st St, New York City, NY 10017",4.3,"Steakhouses, Seafood, Bars",$$$$
1,Park Ave Kitchen by David Burke,(646) 847-4166,https://www.cityguideny.com/linktrack.cfm?id=6...,"514 Lexington Ave 48th St, New York, NY 10017",4.3,"New American, Breakfast & Brunch",$$$
3,The Sea Fire Grill,(212) 935-3785,https://www.cityguideny.com/linktrack.cfm?id=5...,"158 E 48th St, New York, NY 10017",4.5,"Seafood, Steakhouses, Bars",$$$
4,Benjamin Prime,(212) 338-0818,https://www.cityguideny.com/linktrack.cfm?id=6...,"23 E 40th St, New York, NY 10016",4.4,"Seafood, Steakhouses, Bars",$$$$
5,Cafe Fiorello,(212) 595-5330,https://www.cityguideny.com/linktrack.cfm?id=6...,"1900 Broadway, New York, NY 10023",3.6,"Italian, Pizza, American",$$$
6,Redeye Grill,(212) 541-9000,https://www.cityguideny.com/linktrack.cfm?id=6...,"890 7th Ave, New York, NY 10019",3.7,"New American, Seafood, Cocktail Bars",$$$
7,The Palm Restaurant,(212) 333-7256,https://www.cityguideny.com/linktrack.cfm?id=1...,"250 W 50th St, New York, NY 10019",3.6,"Steakhouses, Seafood",$$$
8,Gatsby's Landing,(212) 597-2850,https://www.cityguideny.com/linktrack.cfm?id=6...,"120 West 44th St, New York, NY 10036",4.3,New American,$$$
9,Bond 45,(212) 869-4545,https://www.cityguideny.com/linktrack.cfm?id=6...,"221 W 46th St, New York, NY 10036",3.7,"Italian, Pizza, New American",$$$
10,The Consulate Midtown,(646) 850-1100,https://www.cityguideny.com/linktrack.cfm?id=6...,"44 W 56th St, New York, NY 10019",4.3,"French, New American, Cocktail Bars",$$$$


In [ ]:
# Install the SQLAlchemy library if it is not installed
!sudo apt-get install python3-dev libmysqlclient-dev > /dev/null
!pip install mysqlclient > /dev/null
!sudo pip3 install -U sql_magic > /dev/null
!pip install psycopg2-binary > /dev/null'

# Google colab upgraded the default SQL Alchemy … this break pandas read_sql
# We are downgrading SQL Alchemy for now.
!pip install -U 'sqlalchemy<2.0'

/bin/bash: -c: line 1: unexpected EOF while looking for matching `''
/bin/bash: -c: line 2: syntax error: unexpected end of file
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.3 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.29
    Uninstalling SQLAlchemy-2.0.29:
      Successfully uninstalled SQLAlchemy-2.0.29
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.52 which is incompatible.


In [ ]:
from sqlalchemy import create_engine

In [ ]:
conn_string = 'mysql://{user}:{password}@{host}:{port}/{db}?charset=utf8'.format(
    user='Cocoa',
    password='5xUzs4BPI1g=',
    host = 'jsedocc7.scrc.nyu.edu',
    port     = 3306,
    encoding = 'utf-8',
    db = 'Cocoa'
)
engine = create_engine(conn_string)

In [ ]:
from sqlalchemy import event

# This is for speeding up the insertion into the database schema
@event.listens_for(engine, 'before_cursor_execute')
def receive_before_cursor_execute(conn, cursor, statement, params, context, executemany):
    if executemany:
        cursor.fast_executemany = True

In [ ]:
%reload_ext sql_magic
%config SQL.conn_name = 'engine'

In [ ]:
df_dropped.to_sql(name='food_drink_cgr', # name the table
                   con=engine, # use the connection to MySQL created earlier
                   if_exists='replace', # if the table is already there, replace it
                   index=False, # do not write the index column in the database
                   chunksize=1000, # write 1000 lines at a time
                   method='multi'
)

35

In [ ]:
%%read_sql
show tables;

Query started at 09:19:17 PM UTC; Query executed in 0.00 m

,Tables_in_Cocoa
0,food_drink
1,food_drink_backup
2,food_drink_cgr
3,food_drink_er
4,food_drink_v1
5,food_drink_v2
6,food_drink_v3
7,movies
8,test
9,users_ex


In [ ]:
pd.read_sql("SELECT * FROM food_drink_cgr", con=engine)

,Name,Phone,Website,Location,Rating,Type of Cuisine,Price Point
0,Benjamin Steakhouse,(212) 297-9177,https://www.cityguideny.com/linktrack.cfm?id=1...,"52 E 41st St, New York City, NY 10017",4.3,"Steakhouses, Seafood, Bars",$$$$
1,Park Ave Kitchen by David Burke,(646) 847-4166,https://www.cityguideny.com/linktrack.cfm?id=6...,"514 Lexington Ave 48th St, New York, NY 10017",4.3,"New American, Breakfast & Brunch",$$$
2,The Sea Fire Grill,(212) 935-3785,https://www.cityguideny.com/linktrack.cfm?id=5...,"158 E 48th St, New York, NY 10017",4.5,"Seafood, Steakhouses, Bars",$$$
3,Benjamin Prime,(212) 338-0818,https://www.cityguideny.com/linktrack.cfm?id=6...,"23 E 40th St, New York, NY 10016",4.4,"Seafood, Steakhouses, Bars",$$$$
4,Cafe Fiorello,(212) 595-5330,https://www.cityguideny.com/linktrack.cfm?id=6...,"1900 Broadway, New York, NY 10023",3.6,"Italian, Pizza, American",$$$
5,Redeye Grill,(212) 541-9000,https://www.cityguideny.com/linktrack.cfm?id=6...,"890 7th Ave, New York, NY 10019",3.7,"New American, Seafood, Cocktail Bars",$$$
6,The Palm Restaurant,(212) 333-7256,https://www.cityguideny.com/linktrack.cfm?id=1...,"250 W 50th St, New York, NY 10019",3.6,"Steakhouses, Seafood",$$$
7,Gatsby's Landing,(212) 597-2850,https://www.cityguideny.com/linktrack.cfm?id=6...,"120 West 44th St, New York, NY 10036",4.3,New American,$$$
8,Bond 45,(212) 869-4545,https://www.cityguideny.com/linktrack.cfm?id=6...,"221 W 46th St, New York, NY 10036",3.7,"Italian, Pizza, New American",$$$
9,The Consulate Midtown,(646) 850-1100,https://www.cityguideny.com/linktrack.cfm?id=6...,"44 W 56th St, New York, NY 10019",4.3,"French, New American, Cocktail Bars",$$$$
